In [1]:
from DrissionPage import ChromiumPage, ChromiumOptions

def login_and_save():
    # 配置
    co = ChromiumOptions()
    # 指定用户数据目录（会自动生成）
    co.set_user_data_path(path='BrowserUserData')
    
    # 【关键】指定一个固定端口，确保不会乱开其他浏览器实例
    co.set_local_port(9222)

    # 启动
    page = ChromiumPage(co)
    
    print("正在打开链家...")
    page.get('https://bj.lianjia.com/')
    
    print("------------------------------------------------")
    print("请在弹出的浏览器中，手动完成登录！")
    print("1. 点击右上角登录")
    print("2. 扫码或验证码登录")
    print("3. 【重点】：登录成功后，随便刷新一下页面，确保头像显示正常")
    print("------------------------------------------------")
    
    input("登录成功了吗？成功后请按【回车键】保存并退出...")
    
    # 彻底关闭浏览器，释放文件锁
    page.quit()
    print("✅ 登录信息已保存到 BrowserUserData 文件夹！")

if __name__ == "__main__":
    login_and_save()

正在打开链家...
------------------------------------------------
请在弹出的浏览器中，手动完成登录！
1. 点击右上角登录
2. 扫码或验证码登录
3. 【重点】：登录成功后，随便刷新一下页面，确保头像显示正常
------------------------------------------------
登录成功了吗？成功后请按【回车键】保存并退出...
✅ 登录信息已保存到 BrowserUserData 文件夹！


In [1]:
# ==========================================
# 链家租房数据爬虫 - 终极稳定版 (Jupyter专用)
# ==========================================
from DrissionPage import ChromiumPage, ChromiumOptions
import time
import random
import csv
import os
from datetime import datetime

# --- 配置区 ---
CSV_FILE = "lianjia_data_final.csv"  # 数据保存文件名
USER_DATA_PATH = "BrowserUserData"   # 必须和你Step1设置的文件夹名字完全一样
PORT = 9222                          # 必须和你Step1设置的端口一样

def save_to_csv(data_list):
    """追加写入CSV，防丢失"""
    is_new = not os.path.exists(CSV_FILE)
    # 使用 utf-8-sig 编码，防止Excel打开乱码
    with open(CSV_FILE, mode='a', encoding='utf-8-sig', newline='') as f:
        writer = csv.writer(f)
        if is_new:
            # 写入表头
            writer.writerow(['标题', '价格', '区域', '链接', '抓取时间'])
        writer.writerows(data_list)

def run_spider_jupyter():
    # 1. 初始化浏览器 (继承登录状态)
    co = ChromiumOptions()
    co.set_user_data_path(path=USER_DATA_PATH)
    co.set_local_port(PORT)
    
    # 尽可能模拟真人，禁用自动化特征条
    co.set_argument('--disable-infobars')
    co.set_argument('--no-first-run')
    
    # 启动浏览器对象
    try:
        page = ChromiumPage(co)
    except Exception as e:
        print(f"❌ 浏览器启动失败，请检查是否关闭了Step1的窗口？错误: {e}")
        return

    # 2. 设置爬取范围
    start_page = 1
    end_page = 35  # 爬取前35页 (约1000条)
    
    print(f"🚀 任务开始！目标：第 {start_page} - {end_page} 页")
    print(f"📁 数据将实时保存到: {CSV_FILE}")

    # 3. 循环爬取
    for i in range(start_page, end_page + 1):
        url = f"https://bj.lianjia.com/zufang/pg{i}/"
        print(f"\n[{datetime.now().strftime('%H:%M:%S')}] 正在访问第 {i} 页...")
        
        try:
            page.get(url)
            
            # ---【核心反爬对抗模块】---
            # 检测是否触发了验证码或强制登录
            if "验证" in page.title or "captcha" in page.url:
                print("\n" + "!"*50)
                print("🚨 警告：触发了人机验证！")
                print("👉 请切换到浏览器窗口，手动完成滑块验证！")
                input("✅ 完成验证后，请在这个输入框按【回车键】继续...")
                print("🔄 正在重试本页...")
                page.refresh()
                time.sleep(3)
            # ------------------------

            # 等待数据加载 (最长等5秒)
            try:
                page.wait.ele('.content__list--item', timeout=5)
            except:
                print("⚠️ 本页加载超时或无数据 (可能是空页或网络慢)")

            # 获取所有房源卡片
            items = page.eles('.content__list--item')
            
            # 如果依然找不到元素，可能是被封了，暂停检查
            if not items:
                print(f"❌ 异常：页面代码返回 200 但找不到房源。当前标题：{page.title}")
                # 再次给予手动处理的机会
                input("👉 请检查浏览器状态，解决后按回车重试...")
                page.refresh()
                time.sleep(3)
                items = page.eles('.content__list--item')

            current_page_data = []
            print(f"   - 发现列表元素 {len(items)} 个，开始解析...")

            # 遍历提取
            for item in items:
                try:
                    # 1. 提取标题 (如果没有标题，说明是广告)
                    title_ele = item.ele('.content__list--item--title', timeout=0.5)
                    if not title_ele:
                        # 跳过广告位
                        continue
                    
                    title = title_ele.text.strip()
                    
                    # 2. 提取价格
                    price_ele = item.ele('.content__list--item-price', timeout=0.5)
                    price = price_ele.text.strip() if price_ele else "价格待议"

                    # 3. 提取区域 (可选)
                    loc_ele = item.ele('.content__list--item--des', timeout=0.5)
                    location = loc_ele.text.replace('\n', ' ').strip() if loc_ele else ""

                    # 4. 提取链接
                    link_ele = item.ele('a', timeout=0.5)
                    link = link_ele.link if link_ele else ""

                    # 加入列表
                    current_page_data.append([title, price, location, link, datetime.now()])

                except Exception as e:
                    # 单条数据解析出错不影响整体
                    continue

            # 保存本页数据
            if current_page_data:
                save_to_csv(current_page_data)
                print(f"✅ 第 {i} 页成功保存 {len(current_page_data)} 条有效数据")
            else:
                print("⚠️ 本页没有提取到有效数据。")

            # ---【智能防封等待】---
            # 每爬5页，强制休息长一点 (20-40秒)
            if i % 5 == 0:
                rest_time = random.uniform(20, 40)
                print(f"☕ 已经爬了5页，为了防止封号，休息 {rest_time:.1f} 秒...")
                time.sleep(rest_time)
            else:
                # 普通翻页休息 (3-7秒)
                rest_time = random.uniform(3, 7)
                print(f"⏳ 随机等待 {rest_time:.1f} 秒...")
                time.sleep(rest_time)

        except Exception as e:
            print(f"❌ 第 {i} 页发生未知错误: {e}")
            # 遇到严重错误也不退，尝试下一页
            time.sleep(5)
            continue
            
    print("\n🎉🎉🎉 全部任务结束！请查看 lianjia_data_final.csv 文件。")

# 运行爬虫
if __name__ == "__main__":
    run_spider_jupyter()

🚀 任务开始！目标：第 1 - 35 页
📁 数据将实时保存到: lianjia_data_final.csv

[19:09:15] 正在访问第 1 页...
⚠️ 本页加载超时或无数据 (可能是空页或网络慢)
   - 发现列表元素 30 个，开始解析...
✅ 第 1 页成功保存 26 条有效数据
⏳ 随机等待 5.9 秒...

[19:09:37] 正在访问第 2 页...
⚠️ 本页加载超时或无数据 (可能是空页或网络慢)
   - 发现列表元素 30 个，开始解析...
✅ 第 2 页成功保存 26 条有效数据
⏳ 随机等待 6.3 秒...

[19:10:00] 正在访问第 3 页...
⚠️ 本页加载超时或无数据 (可能是空页或网络慢)
   - 发现列表元素 30 个，开始解析...
✅ 第 3 页成功保存 26 条有效数据
⏳ 随机等待 5.6 秒...

[19:10:22] 正在访问第 4 页...
⚠️ 本页加载超时或无数据 (可能是空页或网络慢)
   - 发现列表元素 30 个，开始解析...
✅ 第 4 页成功保存 26 条有效数据
⏳ 随机等待 3.9 秒...

[19:10:42] 正在访问第 5 页...
⚠️ 本页加载超时或无数据 (可能是空页或网络慢)
   - 发现列表元素 30 个，开始解析...
✅ 第 5 页成功保存 26 条有效数据
☕ 已经爬了5页，为了防止封号，休息 33.6 秒...

[19:11:32] 正在访问第 6 页...
⚠️ 本页加载超时或无数据 (可能是空页或网络慢)
   - 发现列表元素 30 个，开始解析...
✅ 第 6 页成功保存 22 条有效数据
⏳ 随机等待 6.7 秒...

[19:11:55] 正在访问第 7 页...

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
🚨 警告：触发了人机验证！
👉 请切换到浏览器窗口，手动完成滑块验证！
✅ 完成验证后，请在这个输入框按【回车键】继续...
🔄 正在重试本页...
⚠️ 本页加载超时或无数据 (可能是空页或网络慢)
   - 发现列表元素 30 个，开始解析...
✅ 第 7 页成功保存 30 条有效数据
⏳ 随机等待 4.9 秒...

[19:12:37] 正